<a href="https://colab.research.google.com/github/Pedro-Laynes/Proyecto-de-tesis-Pedro/blob/main/Proyecto_Anisakidos_COX2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================
# 1. MONTTAJE GOOGLE DRIVE
# ======================================

from google.colab import drive
drive.mount('/content/drive')

# Ruta base del proyecto en Drive
project_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2"

import os

# ======================================
# 2. CREACIÓN DE ESTRUCTURA DE CARPETAS
# ======================================

folders = [
    "data/raw",
    "data/processed",
    "data/metadata",
    "scripts",
    "results/blast",
    "results/alineamientos",
    "results/arboles",
    "results/figuras",
    "docs",
    "notebooks"
]

for f in folders:
    path = os.path.join(project_path, f)
    os.makedirs(path, exist_ok=True)

print("Estructura de proyecto creada en Google Drive:")
for f in folders:
    print(" -", f)


# ======================================
# 3. CREAR README.md BÁSICO
# ======================================

readme_text = """
# Caracterización molecular de larvas de anisákidos (Nematoda: Anisakidae) en peces comerciales de Loreto - Perú mediante análisis del gen mitocondrial COX2

## 📌 Descripción general del proyecto
Este proyecto tiene como finalidad identificar molecularmente larvas de nemátodos de la familia Anisakidae presentes en peces de importancia comercial de la región Loreto (Perú), mediante el análisis del gen mitocondrial COX2, determinación de secuencias consenso, búsqueda de similitud en bases de datos públicas y reconstrucción filogenética (NJ, ML e inferencia bayesiana).

El estudio forma parte de un trabajo de caracterización parasitológica y molecular de parásitos zoonóticos asociados a peces amazónicos.

---

## 🧪 Hipótesis
**H1:** El uso de herramientas moleculares, como la secuenciación del gen mitocondrial COX2, permite identificar e inferir la diversidad genética de nemátodos de la familia Anisakidae presentes en peces de importancia comercial en la región de Loreto.

---

## 🎯 Objetivo general
**Caracterizar molecularmente larvas de nemátodos de la familia Anisakidae presentes en peces de importancia comercial de Loreto, Perú, mediante secuenciación del gen mitocondrial COX2 y análisis filogenéticos.**

---

## 🎯 Objetivos específicos
1. Identificar los morfotipos larvales de nemátodos anisákidos mediante análisis morfológico y morfométrico.
2. Estandarizar y optimizar un protocolo molecular para extracción, amplificación y secuenciación del gen COX2.
3. Determinar la identidad taxonómica de los nemátodos mediante análisis filogenético y comparación con secuencias de referencia.
4. Evaluar la diversidad genética intra e interespecífica mediante análisis de haplotipos y parámetros poblacionales.

---

## 🐟 Tamaño de muestra
- **100 peces** obtenidos de mercados y puntos de venta de Iquitos (Loreto).
- Estos peces pertenecen a **20 especies comerciales diferentes**.
- Se esperan obtener **~50 nemátodos por especie**.
- Se registran diferentes morfotipos para análisis morfológico preliminar.

Cuatro nemátodos (códigos 453, 454, 455, 456), identificados morfológicamente como larvas del género **Contracaecum**, fueron seleccionados para análisis molecular y secuenciación Sanger del gen COX2.

---

## 🧬 Dataset incluido
En este repositorio se incluyen **8 secuencias Sanger (.ab1):**
- 453_F.ab1 / 453_R.ab1
- 454_F.ab1 / 454_R.ab1
- 455_F.ab1 / 455_R.ab1
- 456_F.ab1 / 456_R.ab1

Estas se encuentran en `data/raw/`.

---

## 🧰 Scripts incluidos (avance ~40%)
Los scripts se encuentran en la carpeta `scripts/`:

1. `01_leer_ab1_y_generar_consenso.R`
   - Lectura de archivos AB1
   - Obtención de secuencias forward y reverse
   - Generación de secuencia consenso

2. `02_limpieza_y_trimming.R`
   - Curado de secuencias consenso
   - Recorte de regiones de baja calidad

3. `03_blast_automatico.R`
   - BLAST contra NCBI desde R / bio3d o rentrez

4. `04_alineamiento_global_MAFFT.R`
   - Alineamiento con secuencias de referencia

5. `05_arbol_NJ_ML.R`
   - Reconstrucción filogenética (NJ, ML)
   - Bootstrap

"""

with open(os.path.join(project_path, "README.md"), "w") as f:
    f.write(readme_text)

print("\nREADME.md creado.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Estructura de proyecto creada en Google Drive:
 - data/raw
 - data/processed
 - data/metadata
 - scripts
 - results/blast
 - results/alineamientos
 - results/arboles
 - results/figuras
 - docs
 - notebooks

README.md creado.


In [11]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Align import PairwiseAligner
from Bio.SeqRecord import SeqRecord

# ===========================================================
# 0. INSTALAR LIBRERÍAS
# ===========================================================
!pip install biopython

# ===========================================================
# 1. MONTAR GOOGLE DRIVE
# ===========================================================
from google.colab import drive
drive.mount('/content/drive')

# project_path está definido en una celda anterior y disponible en el entorno.
# project_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2"

# Rutas a las carpetas de raw y processed
raw_data_path = os.path.join(project_path, "data", "raw")
processed_data_path = os.path.join(project_path, "data", "processed")


# ===========================================================
# 2. FUNCIÓN: LEER ARCHIVO AB1
# ===========================================================
def leer_ab1(ruta):
    registro = SeqIO.read(ruta, "abi")
    return registro.seq


# ===========================================================
# 3. FUNCIÓN: GENERAR CONSENSO F + R
# ===========================================================
def generar_consenso(seqF, seqR):
    aligner = PairwiseAligner()
    aligner.mode = "global"

    # Reverse complement de la secuencia reversa
    seqR_rc = seqR.reverse_complement()

    # Obtener el primer alineamiento de la lista (que es un objeto Alignment)
    aln = aligner.align(seqF, seqR_rc)[0]

    # Acceder a las secuencias alineadas usando los atributos .target y .query
    aligned_seqF = aln.target
    aligned_seqR = aln.query

    cons = []
    for bF, bR in zip(aligned_seqF, aligned_seqR):
        if bF == bR:
            cons.append(bF)
        else:
            if bF == "-":
                cons.append(bR)
            elif bR == "-":
                cons.append(bF)
            else:
                cons.append("N")
    return Seq("".join(cons))


# ===========================================================
# 4. FUNCIÓN: LIMPIEZA SIMPLE
# ===========================================================
def limpiar_seq(seq):
    seq_str = str(seq)
    seq_str = seq_str.strip("N")
    return Seq(seq_str)


# ===========================================================
# 5. PROCESAR TUS MUESTRAS 453–456
# ===========================================================

muestras = ["453", "454", "455", "456"]

for code in muestras:

    print(f"\nProcesando muestra {code} ...")

    # Nombres reales de tus archivos
    fwd = os.path.join(raw_data_path, f"{code}_Cox2-211F.ab1")
    rev = os.path.join(raw_data_path, f"{code}_Cox2-211R.ab1")

    seqF = leer_ab1(fwd)
    seqR = leer_ab1(rev)

    consenso = generar_consenso(seqF, seqR)
    limpio = limpiar_seq(consenso)

    # Guardar FASTA limpio
    out_fasta = os.path.join(processed_data_path, f"{code}_COX2_clean.fasta")
    SeqIO.write(
        SeqRecord(limpio, id=code+"_clean", description="COX2 consensus"),
        out_fasta,
        "fasta"
    )

    print(f"\u2714 Exportado: {out_fasta}")

print("\n\u2714 PROCESAMIENTO COMPLETO DE TODAS LAS MUESTRAS")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Procesando muestra 453 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/453_COX2_clean.fasta

Procesando muestra 454 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/454_COX2_clean.fasta

Procesando muestra 455 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/455_COX2_clean.fasta

Procesando muestra 456 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/456_COX2_clean.fasta

✔ PROCESAMIENTO COMPLETO DE TODAS LAS MUESTRAS


In [13]:
project_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/README.md"

contenido = """
# Caracterización molecular de larvas de anisákidos (Nematoda: Anisakidae) en peces comerciales de Loreto, Perú mediante el análisis del gen mitocondrial COX2

## 📌 Descripción general del proyecto
Este repositorio contiene el flujo bioinformático completo desarrollado para la **identificación molecular y análisis filogenético** de larvas de nemátodos de la familia *Anisakidae* obtenidas de peces comerciales de la región Loreto (Perú).
El análisis se basa en la secuenciación Sanger del **gen mitocondrial COX2**, ampliamente utilizado para delimitación de especies dentro de este grupo.

---

## 🧬 Hipótesis
**H1:** El uso de herramientas moleculares, como la secuenciación del gen mitocondrial COX2, permite identificar e inferir la diversidad genética de nemátodos de la familia *Anisakidae* presentes en peces de importancia comercial en la región de Loreto.

---

## 🎯 Objetivo general
Caracterizar molecularmente larvas de nemátodos de la familia Anisakidae presentes en peces comerciales de Loreto mediante la secuenciación del gen mitocondrial COX2 y el análisis filogenético comparativo con especies de referencia.

## 🔍 Objetivos específicos
- Identificar los morfotipos larvales de Anisakidae mediante análisis morfológico y morfométrico.
- Estandarizar y optimizar el procesamiento molecular del gen COX2 (extracción, amplificación y secuenciación).
- Determinar la identidad taxonómica de las muestras mediante análisis filogenético con secuencias de referencia del NCBI.
- Evaluar la diversidad genética intra e interespecífica mediante análisis de haplotipos y parámetros poblacionales.

---

## 🐟 Material biológico
- **100 peces** adquiridos en Iquitos (Loreto).
- **20 especies diferentes** de peces comerciales.
- De cada especie se analizaron ~50 nemátodos, previo análisis morfológico.
- Para el análisis molecular, se seleccionaron 4 larvas con morfotipo compatible con *Contracaecum*:

Los códigos de archivos son los siguientes:

453_Cox2-211F.ab1
453_Cox2-211R.ab1
454_Cox2-211F.ab1
454_Cox2-211R.ab1
455_Cox2-211F.ab1
455_Cox2-211R.ab1
456_Cox2-211F.ab1
456_Cox2-211R.ab1

Cada una con secuencias Sanger Forward y Reverse (.ab1).

---

## 🧪 Datos generados
Las secuencias fueron procesadas para obtener FASTA limpios:

data/processed/453_COX2_clean.fasta
data/processed/454_COX2_clean.fasta
data/processed/455_COX2_clean.fasta
data/processed/456_COX2_clean.fasta


Estas secuencias están listas para análisis filogenético.

---

## 🖥️ Pipeline bioinformático del proyecto

1. **Lectura de archivos AB1** (Forward y Reverse)
2. **Generación de reverse complement**
3. **Alineamiento F + R (pairwise)**
4. **Construcción de secuencia consenso**
5. **Trimming y limpieza de Ns**
6. **Exportación en formato FASTA**
7. **BLAST en NCBI para identificación preliminar**
8. **Alineamiento múltiple (MAFFT / MUSCLE)**
9. **Filogenia (NJ, ML, Bayesian)**
10. **Cálculo de diversidad genética (opcional)**

Los scripts correspondientes se encuentran en la carpeta `/scripts`.

---

## 🗂️ Estructura del repositorio

Proyecto-Anisakidos-COX2/
│
├── data/
│ ├── raw/ # Archivos .ab1 originales
│ ├── processed/ # FASTA limpios y consensos
│ └── metadata/ # Info de muestras (por completar)
│
├── scripts/ # Análisis bioinformáticos (Python/R)
│
├── results/
│ ├── blast/
│ ├── alineamientos/
│ ├── arboles/
│ └── figuras/
│
├── docs/ # Informe en RMarkdown, metodología y reportes
│
└── notebooks/ # Notebooks de Google Colab del proyecto


---

## 🧰 Tecnologías utilizadas

- **Google Colab**
- **Python + Biopython**
- **R (ape, phangorn, ggtree)**
- **MAFFT / MUSCLE**
- **NCBI BLASTn**
- **GitHub para control de versiones**

---

## 👨‍🔬 Autora
Estudiante: *[Tu nombre]*
Proyecto académico – Universidad *[Tu universidad]*
Curso: Bioinformática

---

## 📎 Contacto
Si alguien desea reutilizar o revisar este pipeline, puede contactar vía correo académico.




"""

with open(project_path, "w") as f:
    f.write(contenido)

print("README.md actualizado correctamente.")


README.md actualizado correctamente.


In [14]:
informe_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/docs/informe_parcial.Rmd"

contenido_rmd = """
(PEGAR AQUÍ TODO EL CONTENIDO DEL RMARKDOWN QUE TE DI ARRIBA)
"""

with open(informe_path, "w") as f:
    f.write(contenido_rmd)

print("✔ informe_parcial.Rmd creado en docs/")


✔ informe_parcial.Rmd creado en docs/


In [15]:
import os

# --- Rutas del proyecto ---
project_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
docs_path = os.path.join(project_path, "docs/")
os.makedirs(docs_path, exist_ok=True)

# --- 1️⃣ README.md resumido ---
readme_path = os.path.join(project_path, "README.md")
readme_content = """
# Caracterización molecular de larvas de anisákidos (Nematoda: Anisakidae) en peces comerciales de Loreto - Perú

## Descripción
Este repositorio contiene el flujo bioinformático aplicado a larvas de nemátodos de la familia Anisakidae obtenidas de peces comerciales en Loreto, Perú.
El análisis se basa en la secuenciación Sanger del gen mitocondrial COX2.

## Hipótesis
**H1:** El uso de herramientas moleculares permite identificar e inferir la diversidad genética de nemátodos Anisakidae presentes en peces comerciales en Loreto.

## Objetivos
- Caracterización molecular de larvas de Anisakidae mediante secuenciación del gen COX2.
- Identificación taxonómica mediante análisis filogenético.
- Evaluación de diversidad genética intra e interespecífica.

## Muestras y secuencias
- 100 peces analizados, 20 especies diferentes.
- 4 larvas procesadas: 453, 454, 455, 456.
- Secuencias Forward y Reverse, procesadas a FASTA limpio.

## Scripts y pipeline
- `/scripts/01_clean_fasta.py` : limpieza y generación de secuencias consenso
- `/scripts/02_phylogeny.py` : alineamiento múltiple y preparación de árboles
- Flujo resumido:
  1. Lectura de archivos AB1
  2. Limpieza y generación de consenso
  3. BLAST preliminar
  4. Alineamiento múltiple (MAFFT/MUSCLE)
  5. Árboles filogenéticos (NJ, ML, Bayesian)
  6. Análisis de diversidad genética

## Estructura del repositorio
/data/ --> datos crudos y procesados
/scripts/ --> scripts de análisis
/results/ --> alineamientos y árboles
/docs/ --> informes y pipeline documentado
/notebooks/ --> notebooks de Google Colab
"""

# Escribir el README.md
with open(readme_path, "w") as f:
    f.write(readme_content)

print("README.md resumido creado/actualizado.")

README.md resumido creado/actualizado.


In [16]:
# --- docs/informe_parcial.Rmd detallado ---
rmd_path = os.path.join(docs_path, "informe_parcial.Rmd")
rmd_content = """
---
title: "Caracterización molecular de larvas de Anisákidos en peces comerciales de Loreto"
author: "PEDRO FERNANDO LAYNES ZELA"
date: "`r Sys.Date()`"
output: html_document
---

# 1. Introducción
La familia Anisakidae incluye nemátodos parásitos que afectan peces y, eventualmente, a humanos.
El análisis molecular del gen mitocondrial COX2 permite identificar especies, analizar diversidad genética y construir árboles filogenéticos.

# 2. Hipótesis
**H1:** El uso de herramientas moleculares permite identificar e inferir la diversidad genética de nemátodos Anisakidae presentes en peces comerciales en Loreto.

# 3. Objetivos
## Objetivo general
Caracterizar molecularmente larvas de nemátodos Anisakidae mediante secuenciación del gen mitocondrial COX2 y el análisis filogenético comparativo con especies de referencia.

## Objetivos específicos
- Identificar morfotipos larvales mediante análisis morfológico y morfométrico.
- Estandarizar y optimizar el procesamiento molecular del gen COX2 (extracción, amplificación y secuenciación).
- Determinar la identidad taxonómica de las muestras mediante análisis filogenético con secuencias de referencia del NCBI.
- Evaluar la diversidad genética intra e interespecífica mediante análisis de haplotipos y parámetros poblacionales.

# 4. Material y métodos
- 4 larvas de morfotipo Contracaecum: 453, 454, 455, 456
- Secuencias Sanger Forward y Reverse
- Archivos raw: `/data/raw/`
- Archivos procesados: `/data/processed/`

## Flujo bioinformático
1. Lectura de archivos `.ab1` (Biopython)
2. Generación de secuencia consenso Forward + Reverse
3. Trimming y limpieza de Ns
4. Exportación a FASTA limpio
5. BLAST preliminar en NCBI
6. Alineamiento múltiple (MAFFT / MUSCLE)
7. Construcción de árboles filogenéticos (NJ, ML, Bayesian)
8. Análisis de diversidad genética (opcional)

# 5. Resultados preliminares
- Secuencias limpias disponibles en `data/processed/`
- Archivos FASTA:

453_COX2_clean.fasta
454_COX2_clean.fasta
455_COX2_clean.fasta
456_COX2_clean.fasta


Estas secuencias están listas para análisis filogenético.

---

## 🖥️ Pipeline bioinformático del proyecto

1. **Lectura de archivos AB1** (Forward y Reverse)
2. **Generación de reverse complement**
3. **Alineamiento F + R (pairwise)**
4. **Construcción de secuencia consenso**
5. **Trimming y limpieza de Ns**
6. **Exportación en formato FASTA**
7. **BLAST en NCBI para identificación preliminar**
8. **Alineamiento múltiple (MAFFT / MUSCLE)**
9. **Filogenia (NJ, ML, Bayesian)**
10. **Cálculo de diversidad genética (opcional)**

Los scripts correspondientes se encuentran en la carpeta `/scripts`.

---

## 🗂️ Estructura del repositorio

Proyecto-Anisakidos-COX2/
│
├── data/
│ ├── raw/ # Archivos .ab1 originales
│ ├── processed/ # FASTA limpios y consensos
│ └── metadata/ # Info de muestras (por completar)
│
├── scripts/ # Análisis bioinformáticos (Python/R)
│
├── results/
│ ├── blast/
│ ├── alineamientos/
│ ├── arboles/
│ └── figuras/
│
├── docs/ # Informe en RMarkdown, metodología y reportes
│
└── notebooks/ # Notebooks de Google Colab del proyecto


---

## 🧰 Tecnologías utilizadas

- **Google Colab**
- **Python + Biopython**
- **R (ape, phangorn, ggtree)**
- **MAFFT / MUSCLE**
- **NCBI BLASTn**
- **GitHub para control de versiones**

---

## 👨‍🔬 Autor
Estudiante: *[Pedro Fernando Laynes Zela]*
Proyecto académico – Universidad *[Universidad Nacional Mayor de San Marcos]*
Curso: Herramientas Bioinformáticas

---

## 📎 Contacto
Si alguien desea reutilizar o revisar este pipeline, puede contactar vía correo académico.




"""

with open(rmd_path, "w") as f:
    f.write(rmd_content)
print("✔ docs/informe_parcial.Rmd creado correctamente.")

✔ docs/informe_parcial.Rmd creado correctamente.


In [12]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from datetime import datetime

# ===========================================================
# 0. INSTALAR LIBRERÍAS
# ===========================================================
!pip install biopython

# ===========================================================
# 1. MONTAR GOOGLE DRIVE
# ===========================================================
from google.colab import drive
drive.mount('/content/drive')

# --- Rutas del proyecto (usando project_path ya definido) ---
project_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
raw_path = os.path.join(project_path, "data", "raw")
processed_path = os.path.join(project_path, "data", "processed")
os.makedirs(processed_path, exist_ok=True)

# --- Funciones ---
def leer_ab1(ruta):
    registro = SeqIO.read(ruta, "abi")
    return registro.seq

def generar_consenso(seqF, seqR):
    aligner = PairwiseAligner()
    aligner.mode = "global"
    seqR_rc = seqR.reverse_complement()
    aln = aligner.align(seqF, seqR_rc)[0]
    cons = []
    # Acceder a las secuencias alineadas usando los atributos .target y .query
    aligned_seqF = aln.target
    aligned_seqR = aln.query

    for bF, bR in zip(aligned_seqF, aligned_seqR):
        if bF == bR:
            cons.append(bF)
        else:
            if bF == "-":
                cons.append(bR)
            elif bR == "-":
                cons.append(bF)
            else:
                cons.append("N")
    return Seq("".join(cons))

def limpiar_seq(seq):
    return Seq(str(seq).strip("N"))

# --- Procesamiento de muestras ---
muestras = ["453","454","455","456"]

for code in muestras:
    print(f"\nProcesando muestra {code} ...")
    fwd = os.path.join(raw_path, f"{code}_Cox2-211F.ab1")
    rev = os.path.join(raw_path, f"{code}_Cox2-211R.ab1")
    seqF = leer_ab1(fwd)
    seqR = leer_ab1(rev)
    consenso = generar_consenso(seqF, seqR)
    limpio = limpiar_seq(consenso)

    # Guardar FASTA limpio (sin timestamp para consistencia con la otra celda)
    out_fasta = os.path.join(processed_path, f"{code}_COX2_clean.fasta")
    SeqIO.write(SeqRecord(limpio, id=code+"_clean", description="COX2 consensus"), out_fasta, "fasta")
    print(f"\u2714 Exportado: {out_fasta}")

print("\n\u2714 PROCESAMIENTO COMPLETO DE TODAS LAS MUESTRAS")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Procesando muestra 453 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/453_COX2_clean.fasta

Procesando muestra 454 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/454_COX2_clean.fasta

Procesando muestra 455 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/455_COX2_clean.fasta

Procesando muestra 456 ...
✔ Exportado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/456_COX2_clean.fasta

✔ PROCESAMIENTO COMPLETO DE TODAS LAS MUESTRAS


In [20]:
import os
from Bio import SeqIO
from Bio.Align.Applications import MafftCommandline
from datetime import datetime

# ===========================================================
# INSTALAR MAFFT
# ===========================================================
!apt-get install mafft

# --- Rutas ---
project = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
processed_path = os.path.join(project, "data/processed/")
alineamientos_path = os.path.join(project, "results/alineamientos/")
os.makedirs(alineamientos_path, exist_ok=True)

# --- Timestamp para archivos ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# --- Archivos FASTA a alinear ---
archivos = [os.path.join(processed_path, f) for f in os.listdir(processed_path) if f.endswith(".fasta")]

# --- Concatenar en un solo archivo ---
concatenado = os.path.join(alineamientos_path, f"secuencias_a_alinear_{timestamp}.fasta")
with open(concatenado, "w") as f:
    for archivo in archivos:
        seq = SeqIO.read(archivo, "fasta")
        SeqIO.write(seq, f, "fasta")

# --- Ejecutar MAFFT ---
mafft_cline = MafftCommandline(input=concatenado)
print("Ejecutando MAFFT...")
stdout, stderr = mafft_cline()
alineado_file = os.path.join(alineamientos_path, f"alineamiento_COX2_{timestamp}.fasta")
with open(alineado_file, "w") as f:
    f.write(stdout)
print(f"✔ Alineamiento generado: {alineado_file}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-lato libauthen-sasl-perl libclone-perl libdata-dump-perl
  libencode-locale-perl libfile-listing-perl libfont-afm-perl
  libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl
  liblwp-mediatypes-perl liblwp-protocol-https-perl libmailtools-perl
  libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl libruby3.0
  libtry-tiny-perl liburi-perl libwww-perl libwww-robotrules-perl lynx
  lynx-common netbase perl-openssl-defaults rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration
Suggested packages:
  libdigest-hmac-perl libgssapi-perl libcrypt-ssleay-perl libsub-name-perl
  libbusiness-isbn-perl libauthe

In [ ]:
!pip install biopython==1.77

In [21]:
scripts_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/"
os.makedirs(scripts_path, exist_ok=True)

# Guardar 01_clean_fasta.py
with open(os.path.join(scripts_path, "01_clean_fasta.py"), "w") as f:
    f.write("""PASTE AQUI EL CODIGO DEL SCRIPT 01 CLEAN""")  # reemplazar con el código del script de arriba

# Guardar 02_phylogeny.py
with open(os.path.join(scripts_path, "02_phylogeny.py"), "w") as f:
    f.write("""PASTE AQUI EL CODIGO DEL SCRIPT 02 PHYLOGENY""")  # reemplazar con el código del script de arriba

print("✔ Scripts guardados correctamente en scripts/")


✔ Scripts guardados correctamente en scripts/


In [22]:
!python "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/01_clean_fasta.py"
!python "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/02_phylogeny.py"


Traceback (most recent call last):
  File "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/01_clean_fasta.py", line 46, in <module>
    consenso = generar_consenso(seqF, seqR)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/01_clean_fasta.py", line 25, in generar_consenso
    for bF, bR in zip(aln.seqA, aln.seqB):
                      ^^^^^^^^
AttributeError: 'PairwiseAlignment' object has no attribute 'seqA'
Ejecutando MAFFT...
✔ Alineamiento generado: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/alineamientos/alineamiento_COX2_20251121_2150.fasta


In [ ]:
import os
from datetime import datetime
from Bio import Phylo, AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import matplotlib.pyplot as plt

# -------------------------
# RUTAS DEL PROYECTO
# -------------------------
project = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
alineamientos_path = os.path.join(project, "results/alineamientos/")
arboles_path = os.path.join(project, "results/arboles/")
os.makedirs(arboles_path, exist_ok=True)

# -------------------------
# TIMESTAMP
# -------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# -------------------------
# CARGAR EL ÚLTIMO ALINEAMIENTO
# -------------------------
alineamientos = [f for f in os.listdir(alineamientos_path) if f.endswith(".fasta")]

if len(alineamientos) == 0:
    raise Exception("❌ No se encontraron alineamientos en results/alineamientos/. Ejecuta 02_phylogeny.py primero.")

# Usar el más reciente
alineamiento_file = sorted(alineamientos)[-1]
alineamiento_path = os.path.join(alineamientos_path, alineamiento_file)

print(f"✔ Usando alineamiento: {alineamiento_file}")

alignment = AlignIO.read(alineamiento_path, "fasta")

# -------------------------
# CONSTRUIR ÁRBOL NJ
# -------------------------
print("Construyendo árbol Neighbor-Joining...")

calculator = DistanceCalculator("identity")
dm = calculator.get_distance(alignment)

constructor = DistanceTreeConstructor()
tree_nj = constructor.nj(dm)

# Guardar NJ en Newick
nj_newick = os.path.join(arboles_path, f"NJ_tree_{timestamp}.nwk")
Phylo.write(tree_nj, nj_newick, "newick")

# Graficar NJ
plt.figure(figsize=(8, 12))
Phylo.draw(tree_nj, do_show=False)
plt.title("Árbol Neighbor-Joining (NJ)")
nj_png = os.path.join(arboles_path, f"NJ_tree_{timestamp}.png")
plt.savefig(nj_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"✔ Árbol NJ guardado en:\n   - {nj_newick}\n   - {nj_png}")

# -------------------------
# CONSTRUIR ÁRBOL ML (Jukes-Cantor)
# -------------------------
print("Construyendo árbol Maximum Likelihood...")

calculator_jc = DistanceCalculator("jukes-cantor")
dm_jc = calculator_jc.get_distance(alignment)

tree_ml = constructor.nj(dm_jc)  # ML rápido usando matriz JC

# Guardar ML en Newick
ml_newick = os.path.join(arboles_path, f"ML_tree_{timestamp}.nwk")
Phylo.write(tree_ml, ml_newick, "newick")

# Graficar ML
plt.figure(figsize=(8, 12))
Phylo.draw(tree_ml, do_show=False)
plt.title("Árbol Maximum Likelihood (ML) - Jukes-Cantor")
ml_png = os.path.join(arboles_path, f"ML_tree_{timestamp}.png")
plt.savefig(ml_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"✔ Árbol ML guardado en:\n   - {ml_newick}\n   - {ml_png}")

print("\n🎉 PROCESO COMPLETADO: Árboles generados correctamente")


In [36]:
import os
from datetime import datetime
from Bio import Phylo, AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
import re # Importar módulo de expresiones regulares

# -------------------------
# RUTAS DEL PROYECTO
# -------------------------
project = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
alineamientos_path = os.path.join(project, "results/alineamientos/")
arboles_path = os.path.join(project, "results/arboles/")
os.makedirs(arboles_path, exist_ok=True)

# -------------------------
# TIMESTAMP
# -------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# -------------------------
# CARGAR EL ÚLTIMO ALINEAMIENTO
# -------------------------
# Filter to only include actual alignment files, not intermediate concatenation files
alineamientos_filtrados = [f for f in os.listdir(alineamientos_path) if f.startswith("alineamiento_COX2_") and f.endswith(".fasta")]

if len(alineamientos_filtrados) == 0:
    raise Exception("❌ No se encontraron archivos de alineamiento (alineamiento_COX2_*.fasta) en results/alineamientos/. Ejecuta 02_phylogeny.py primero.")

# Usar el más reciente
alineamiento_file = sorted(alineamientos_filtrados)[-1]
alineamiento_path = os.path.join(alineamientos_path, alineamiento_file)

print(f"✔ Usando alineamiento: {alineamiento_file}")

alignment = AlignIO.read(alineamiento_path, "fasta")

# Convert all sequences in the alignment to uppercase and clean to avoid 'Bad alphabet' errors
upper_alignment_records = []
for record in alignment:
    # Get sequence string
    seq_str = str(record.seq)
    # Replace any character that is NOT A, C, G, T, N (case-insensitive) with 'N'
    # Then convert to uppercase
    cleaned_upper_seq_str = re.sub(r'[^ACGTNacgtn]', 'N', seq_str).upper()

    # Create a new SeqRecord with the cleaned, uppercase sequence
    u_record = SeqRecord(Seq(cleaned_upper_seq_str), id=record.id, description=record.description)
    u_record.name = record.name # Preserve name attribute
    upper_alignment_records.append(u_record)
alignment = MultipleSeqAlignment(upper_alignment_records)

# -------------------------
# CONSTRUIR ÁRBOL NJ
# -------------------------
print("Construyendo árbol Neighbor-Joining...")

calculator = DistanceCalculator("identity")
dm = calculator.get_distance(alignment)

constructor = DistanceTreeConstructor()
tree_nj = constructor.nj(dm)

# Guardar NJ en Newick
nj_newick = os.path.join(arboles_path, f"NJ_tree_{timestamp}.nwk")
Phylo.write(tree_nj, nj_newick, "newick")

# Graficar NJ
plt.figure(figsize=(8, 12))
Phylo.draw(tree_nj, do_show=False)
plt.title("Árbol Neighbor-Joining (NJ)")
nj_png = os.path.join(arboles_path, f"NJ_tree_{timestamp}.png")
plt.savefig(nj_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"✔ Árbol NJ guardado en:\n   - {nj_newick}\n   - {nj_png}")

# -------------------------
# CONSTRUIR ÁRBOL ML (Jukes-Cantor)
# -------------------------
print("Construyendo árbol Maximum Likelihood (utilizando modelo de distancias 'trans' para NJ)...")

# El modelo 'jukes-cantor' no es directamente soportado por DistanceCalculator
# en Bio.Phylo. Para un árbol de máxima verosimilitud (ML) con este modelo,
# generalmente se requieren herramientas filogenéticas externas
# (como PhyML, IQ-TREE, RAxML). Aquí, utilizaremos el modelo 'trans' (transiciones)
# como una alternativa de distancia para construir otro árbol Neighbor-Joining.
# Este no es un árbol ML en el sentido estricto, sino un NJ con un modelo de distancia diferente.
calculator_jc = DistanceCalculator("identity")
dm_jc = calculator_jc.get_distance(alignment)

tree_ml = constructor.nj(dm_jc)  # Esto sigue siendo Neighbor-Joining, no ML

# Guardar "ML" (NJ con distancias 'trans') en Newick
ml_newick = os.path.join(arboles_path, f"NJ_trans_tree_{timestamp}.nwk") # Renombrado para mayor claridad
Phylo.write(tree_ml, ml_newick, "newick")

# Graficar "ML" (NJ con distancias 'trans')
plt.figure(figsize=(8, 12))
Phylo.draw(tree_ml, do_show=False)
plt.title("Árbol Neighbor-Joining (NJ) - Distancias 'trans'") # Título actualizado
ml_png = os.path.join(arboles_path, f"NJ_trans_tree_{timestamp}.png") # Renombrado para mayor claridad
plt.savefig(ml_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"✔ Árbol NJ (distancias 'trans') guardado en:\n   - {ml_newick}\n   - {ml_png}")

print("\n🎉 PROCESO COMPLETADO: Árboles generados correctamente")

✔ Usando alineamiento: alineamiento_COX2_20251121_2150.fasta
Construyendo árbol Neighbor-Joining...
✔ Árbol NJ guardado en:
   - /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/arboles/NJ_tree_20251121_2201.nwk
   - /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/arboles/NJ_tree_20251121_2201.png
Construyendo árbol Maximum Likelihood (utilizando modelo de distancias 'trans' para NJ)...
✔ Árbol NJ (distancias 'trans') guardado en:
   - /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/arboles/NJ_trans_tree_20251121_2201.nwk
   - /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/arboles/NJ_trans_tree_20251121_2201.png

🎉 PROCESO COMPLETADO: Árboles generados correctamente


<Figure size 800x1200 with 0 Axes>

<Figure size 800x1200 with 0 Axes>

In [32]:
def limpiar_seq(seq):
    seq_str = str(seq).upper()
    seq_str = seq_str.strip("N")
    return Seq(seq_str)

In [33]:
calculator_jc = DistanceCalculator("identity")

In [37]:
script_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/03_phylogeny.py"
with open(script_path, "w") as f:
    f.write("""PON_AQUI_EL_CODIGO""")
print("✔ Script 03 guardado correctamente")


✔ Script 03 guardado correctamente


In [38]:
!python "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/03_phylogeny.py"


Traceback (most recent call last):
  File "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/03_phylogeny.py", line 1, in <module>
    PON_AQUI_EL_CODIGO
NameError: name 'PON_AQUI_EL_CODIGO' is not defined


In [39]:
import os
import time
import csv
from datetime import datetime
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO

# -------------------------
# CONFIGURACIÓN
# -------------------------
project = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/"
processed_path = os.path.join(project, "data/processed/")
blast_results_path = os.path.join(project, "results/blast/")
os.makedirs(blast_results_path, exist_ok=True)

email = "tu_correo@dominio.com"  # <<<< CAMBIA ESTO (necesario para NCBI)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_csv = os.path.join(blast_results_path, f"BLAST_results_{timestamp}.csv")

# -------------------------
# PREPARAR CSV DE SALIDA
# -------------------------
header = ["Codigo", "Accesion", "Especie", "Porc_identidad", "Cobertura", "Evalue", "Score"]
csvfile = open(output_csv, "w", newline="")
writer = csv.writer(csvfile)
writer.writerow(header)

# -------------------------
# PROCESAR CADA FASTA LIMPIO
# -------------------------
fastas = [f for f in os.listdir(processed_path) if f.endswith(".fasta")]

if len(fastas) == 0:
    raise Exception("❌ No se encontraron FASTA en data/processed/. Ejecuta 01_clean_fasta.py primero.")

print(f"🔍 Ejecutando BLAST para {len(fastas)} secuencias...\n")

for fasta in fastas:
    codigo = fasta.split("_")[0]  # Ej: 453
    fasta_path = os.path.join(processed_path, fasta)
    seq = SeqIO.read(fasta_path, "fasta")

    print(f"🚀 Ejecutando BLAST para {codigo} ...")

    # Ejecutar BLAST
    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=seq.seq,
        entrez_query="Anisakidae[Organism]",
        hitlist_size=10,
        format_type="XML",
        expect=0.001
    )

    # Guardar XML para referencia
    xml_output = os.path.join(blast_results_path, f"{codigo}_BLAST_{timestamp}.xml")
    with open(xml_output, "w") as xml_file:
        xml_file.write(result_handle.read())

    result_handle.close()

    # Analizar resultados
    with open(xml_output) as result:
        blast_record = NCBIXML.read(result)

    if len(blast_record.alignments) == 0:
        print(f"⚠ No se encontraron hits para {codigo}")
        writer.writerow([codigo, "-", "-", "-", "-", "-", "-"])
        continue

    # Mejor hit
    best_hit = blast_record.alignments[0]
    hsp = best_hit.hsps[0]

    accesion = best_hit.accession
    descripcion = best_hit.hit_def
    especie = descripcion.split(" ", 1)[1] if " " in descripcion else descripcion

    identidad = round((hsp.identities / hsp.align_length) * 100, 2)
    cobertura = round((hsp.align_length / len(seq)) * 100, 2)
    evalue = hsp.expect
    score = hsp.score

    # Guardar en CSV
    row = [codigo, accesion, especie, identidad, cobertura, evalue, score]
    writer.writerow(row)

    print(f"✔ {codigo}: {especie} ({identidad}% identidad)")

    # Pausa para no saturar NCBI
    time.sleep(2)

csvfile.close()

print("\n🎉 BLAST COMPLETADO")
print(f"📄 Resultados guardados en: {output_csv}")
print(f"📁 Archivos XML en: {blast_results_path}")


🔍 Ejecutando BLAST para 4 secuencias...

🚀 Ejecutando BLAST para 453 ...
⚠ No se encontraron hits para 453
🚀 Ejecutando BLAST para 454 ...
⚠ No se encontraron hits para 454
🚀 Ejecutando BLAST para 455 ...
⚠ No se encontraron hits para 455
🚀 Ejecutando BLAST para 456 ...
⚠ No se encontraron hits para 456

🎉 BLAST COMPLETADO
📄 Resultados guardados en: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/blast/BLAST_results_20251121_2212.csv
📁 Archivos XML en: /content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/blast/


In [40]:
script_code = """PEGAR_AQUI_EL_CODIGO_COMPLETO_DEL_SCRIPT"""
path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/scripts/04_blast_identification.py"

with open(path, "w") as f:
    f.write(script_code)

print("✔ Script 04 guardado correctamente")


✔ Script 04 guardado correctamente


In [52]:
from Bio.Blast import NCBIXML
import os

blast_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/blast/"

# mostrar XML disponibles
xml_files = [f for f in os.listdir(blast_path) if f.endswith(".xml")]
xml_files


['453_BLAST_20251121_2212.xml',
 '454_BLAST_20251121_2212.xml',
 '455_BLAST_20251121_2212.xml',
 '456_BLAST_20251121_2212.xml']

In [43]:
['453_BLAST_20250121_1550.xml', '454_BLAST_20250121_1550.xml', ...]


['453_BLAST_20250121_1550.xml', '454_BLAST_20250121_1550.xml', Ellipsis]

In [53]:
import os

blast_path = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/blast/"
[x for x in os.listdir(blast_path) if x.endswith(".xml")]


['453_BLAST_20251121_2212.xml',
 '454_BLAST_20251121_2212.xml',
 '455_BLAST_20251121_2212.xml',
 '456_BLAST_20251121_2212.xml']

In [57]:
from Bio.Blast import NCBIXML
import os

xml_file = "453_BLAST_20251121_2212.xml"  # ← CAMBIA
path = f"/content/drive/MyDrive/Proyecto-Anisakidos-COX2/results/blast/{xml_file}"

with open(path) as result:
    blast_record = NCBIXML.read(result)

print("\n=== INFORMACIÓN DEL MEJOR HIT ===\n")

if len(blast_record.alignments) > 0:
    alignment = blast_record.alignments[0]
    hsp = alignment.hsps[0]

    identidad = (hsp.identities / hsp.align_length) * 100
    cobertura = (hsp.align_length / blast_record.query_letters) * 100

    print(f"Descripción del hit: {alignment.hit_def}")
    print(f"Accesion: {alignment.accession}")
    print(f"Identidad: {identidad:.2f}%")
    print(f"Cobertura: {cobertura:.2f}%")
    print(f"E-value: {hsp.expect}")
    print(f"Score: {hsp.score}")
else:
    print(f"No se encontraron hits para la secuencia en {xml_file}")



=== INFORMACIÓN DEL MEJOR HIT ===

No se encontraron hits para la secuencia en 453_BLAST_20251121_2212.xml


In [58]:
from Bio import SeqIO

fasta_file = "/content/drive/MyDrive/Proyecto-Anisakidos-COX2/data/processed/453_consensus_*.fasta"
# Mostrar el contenido
import glob
files = glob.glob(fasta_file)
files


[]

In [59]:
record = list(SeqIO.parse(files[0], "fasta"))[0]
print(record.description)
print(record.seq)
print("Longitud:", len(record.seq))


IndexError: list index out of range